## 동시성을 위해 스레드가 필요한 경우에는 ThreadpoolExecutor를 사용하라

**Thread와 Queue를 사용한 접근 방법들의 장점을 조합하여 병령 I/O문제를 해결한다.**

In [ ]:
#함수를 실행기에 제출하여 팬아웃한다.
#제출받은 함수를 별도의 스레드에서 수행
ALIVE = '*'
EMPTY = '-'

class Grid:
    ...

class LockingGrid(Grid):
    ...

def count_neighbors(y, x, get):
    ...

def game_logic(state, neighbors):
    ...
    data = my_socket.recv(100) #여기서 블로킹 I/O를 수행한다.
    ...

def stet_cell(y,x, get,set):
    state = get(y,x)
    neighbors = count_neighbors(y,x,get)
    next_state = game_logic(state, neighbors)
    set(y, x, next_state)

In [ ]:
#팬인하기 위한 모든 작업의 결과를 기다린다.
from concurrent.futures import ThreadPoolExecutor

def simulate_pool(pool, grid):
    next_grid = LockingGrid(grid.height, grid.width)
    futures = []
    for y in range(grid.height):
        for x in range(grid.width):
            args = (y,x, grid.get, next_grid.set)
            future = pool.submit(step_cell, *args)#팬 아웃

    for future in futures:
        future.result() #팬인

    return next_grid

In [ ]:
#사용할 스레드를 미리 할당하여 simulate_pool을 실행할 때마다 스레드를 시작하는데 비용이 들지 않는다.
#병렬 I/O 문제를 처리하기 위해서 Thread를 별 생각없이 사용하면 메모리 부족 현상이 일어날 수 있기에
#pool 사용할 스레드 최대 개수 지정하여 해결(max_workers 파라미터)

class ColumnPrinter:
    ...

grid = LockingGrid(5,9)
grid.set(0,3,ALIVE)
grid.set(1,4,ALIVE)
grid.set(2,2,ALIVE)
grid.set(2,3,ALIVE)
grid.set(2, 4,ALIVE)

columns = ColumnPrinter()
with ThreadPoolExecutor(max_workers=10) as pool:
    for i in range(5):
        columns.append(str(grid))
        grid = simulate_pool(pool, grid)

print(columns)

In [ ]:
#스레드를 실행하는 중에 발생한 예외를 자동으로 전파

def game_logic(state, neighbors):
    ...
    raise OSError('I/O 문제 발생')
    ...

with ThreadPoolExecutor(max_workers=10) as pool:
    task = pool.submit(game_logic, ALIVE, 3)
    task.result()

"""
Traceback...
OSError: I/O문제 발생
"""

### Summary

- ThreadPoolExecutor를 사용하여 한정된 리팩터링만으로 간단한 I/O병렬성을 활성화할 수 있고, 동시성을 팬아웃해야 하는 경우에 발생하는 스레드 시작 비용을 쉽게 줄일 수 있다.

- ThreadPoolExecutor를 사용하면 스레드를 직접 사용할 때 발생할 수 있는 잠재적인 메모리 낭비 문제를 없애주면 max_workers의 개수를 미리 지정해야하므로 I/O 병렬성을 제한한다.